In [1]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from PIL import Image

In [2]:
def calculate_psnr(imageA, imageB):
    # Calculate the PSNR between two images. The images must be of the same dimension.
    mse = np.mean((imageA - imageB) ** 2)
    if mse == 0:
        return float('inf')
    MAX_PIXEL_VALUE = 255.0
    return 20 * np.log10(MAX_PIXEL_VALUE / np.sqrt(mse))

def calculate_mse(imageA, imageB):
    # Calculate the MSE between two images. The images must be of the same dimension.
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def calculate_ssim(imageA, imageB):
    # Calculate the SSIM between two images. The images must be of the same dimension.
    return ssim(imageA, imageB, multichannel=True)

def resize_image(image, target_size):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

In [3]:
def get_metrics(folder_name):
    psnr = []
    mse = []
    ssim_list = []
    for i in range(1, 29):

        # Load your images
        imageA = cv2.imread('../plot_images/'+ str(i)+'.png')
        imageB = cv2.imread('../results/'+folder_name+'/'+ str(i)+'.png')

        # Resize images to be the same size
        target_size = (min(imageA.shape[1], imageB.shape[1]), min(imageA.shape[0], imageB.shape[0]))
        imageA_resized = resize_image(imageA, target_size)
        imageB_resized = resize_image(imageB, target_size)

        # Convert images to grayscale for SSIM
        grayA = cv2.cvtColor(imageA_resized, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(imageB_resized, cv2.COLOR_BGR2GRAY)

        # Calculate metrics
        psnr_value = calculate_psnr(imageA_resized, imageB_resized)
        mse_value = calculate_mse(imageA_resized, imageB_resized)
        ssim_value = calculate_ssim(grayA, grayB)

        psnr.append(psnr_value)
        mse.append(mse_value)
        ssim_list.append(ssim_value)
    
    psnr_metric = sum(psnr) / len(psnr)
    mse_metric = sum(mse) / len(mse)
    ssim_metric = sum(ssim_list) / len(ssim_list)
    return psnr_metric, mse_metric, ssim_metric

In [4]:
for folder in [
    # 'kdtree_contrast', 
    # 'euclidean', 
    'cosine'
]:
    psnr_metric, mse_metric, ssim_metric = get_metrics(folder)
    print(f"Folder Name: {folder}")
    print(f"Average PSNR: {psnr_metric}")
    print(f"Average MSE: {mse_metric}")
    print(f"Average SSIM: {ssim_metric}")

Folder Name: cosine
Average PSNR: 35.09663749067515
Average MSE: 25825.421013657742
Average SSIM: 0.545552415204156
